In [35]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [36]:
from financial.momentum.models.kerasAdvanced import KerasAdvancedModelFactory
from financial.momentum.experiment.modelExperiment import ModelExperimentFactory
import os
from dotenv import load_dotenv
from financial.momentum.utilities import find_dotenv
from financial.momentum.utilities import metrics
import financial.data as fd
from financial.io.file.cache import FileCache
import pandas as pd
from financial.lab.experiment import Experiment
from sklearn.metrics import r2_score
from financial.model import Model

import tensorflow as tf
from keras import backend as K
import gc

def reset_gpu():
    K.clear_session()
    tf.keras.backend.clear_session()
    gc.collect()

In [37]:
load_dotenv(dotenv_path=find_dotenv())

cache = os.environ["CACHE"]
model_path = os.environ["MODEL"]
print(model_path)
print(os.environ["DATA"])
datastore = fd.CachedDataStore(path=os.environ["DATA"], cache=FileCache(cache_path=cache+"/"))
print(datastore) 

/home/manidmt/TFG/OTRI/models/keras
/home/manidmt/TFG/OTRI/data
CachedDataStore with 960 data sources [cache stats: {'size': 0, 'hit': 0, 'miss': 0, 'write': 0, 'read': 0, 'update': 0}]


In [38]:
ticker = "MSFT"
start_date = "1990-01-01"
end_date = "2025-06-30"

In [39]:
data = datastore.get_data(ticker, start_date, end_date)
print(data)
target = data[90 + 20:]
print(target.size)

Date
1990-01-02      0.377807
1990-01-03      0.379936
1990-01-04      0.391110
1990-01-05      0.381532
1990-01-08      0.387385
                 ...    
2025-06-24    490.109985
2025-06-25    492.269989
2025-06-26    497.450012
2025-06-27    495.940002
2025-06-30    497.410004
Name: MSFT, Length: 8939, dtype: float64
8829


In [40]:
factory = KerasAdvancedModelFactory()

In [41]:
from itertools import product

# Arquitecturas a probar
architectures = ["rnn", "lstm", "cnn", "cnn2d", "transformer"]

# Hiperparámetros a explorar
layer_option = [64, 32]
activation = ["relu"]
batch_size = 32
epochs = 10

def ticker_single_models(ticker):

    data = datastore.get_data(ticker, start_date, end_date)    
    target = data[90 + 20:]
    
    for arch in architectures:
        name = f"{arch}_{ticker}_{end_date[:4]}_single"

        try:
            if Model.from_file(name, path=os.environ["MODEL"]):
                print(f"⚠️ Modelo {name} ya existe. Saltando...")
                continue
        except FileNotFoundError:
            print(f"📂 Modelo {name} no existe. Procediendo a entrenar...")

        config = {
            "mode": "global",  
            "datastore": datastore,
            "ticker": ticker,
            "model_factory": factory,
            "name": name,
            "start_year": start_date,
            "end_year": end_date,
            "lookahead": 20,
            "horizon": 90,
            "model_params": {
                "architecture": arch,
                "topology": {
                    "layers": layer_option,
                    "activation": {
                        "hidden": activation,
                        "output": "linear"
                    }
                },
                "optimization": {
                    "optimizer": "adam",
                    "loss": "mean_squared_error",
                    "metrics": ["mae"],
                    "epochs": epochs,
                    "batch_size": batch_size,
                    "validation_split": 0.1
                }
            }
        }

        try:
            print("RUNNING MODEL:")
            experiment = ModelExperimentFactory.create_experiment(config)
            experiment.run()
            print("METRICS:")
            preds = experiment.reconstruct_absolute_predictions_from_relative()
            metrics(experiment, preds, target, os.environ["MODEL"])
            reset_gpu()
        except Exception as e:
            print(f"Falló el modelo {name}: {e}")

In [ ]:
ticker_single_models("MSFT")

📂 Modelo rnn_MSFT_2025_single no existe. Procediendo a entrenar...
RUNNING MODEL:
Architecture: rnn
X_train before fit: (6211, 90, 1), y_train: (6211, 1)
Number of 6211 samples, 90 timesteps, 1 features
Epoch 1/300
49/49 ━━━━━━━━━━━━━━━━━━━━ 2s 29ms/step - accuracy: 0.0036 - loss: 0.0164
Epoch 2/300
49/49 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step - accuracy: 0.0030 - loss: 0.0077
Epoch 3/300
49/49 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step - accuracy: 0.0022 - loss: 0.0078
Epoch 4/300
49/49 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step - accuracy: 0.0033 - loss: 0.0076
Epoch 5/300
49/49 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step - accuracy: 0.0025 - loss: 0.0073
Epoch 6/300
49/49 ━━━━━━━━━━━━━━━━━━━━ 1s 30ms/step - accuracy: 0.0020 - loss: 0.0075
Epoch 7/300
49/49 ━━━━━━━━━━━━━━━━━━━━ 1s 30ms/step - accuracy: 0.0028 - loss: 0.0073
Epoch 8/300
49/49 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step - accuracy: 0.0030 - loss: 0.0075
Epoch 9/300
49/49 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step - accuracy: 0.0032 - loss: 0.0073
Epoch 10/300
49/49 ━━━━

195/195 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step
83/83 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step
Architecture: rnn
X_train before fit: (6463, 90, 1), y_train: (6463, 1)
Number of 6463 samples, 90 timesteps, 1 features
Epoch 1/300
51/51 ━━━━━━━━━━━━━━━━━━━━ 2s 29ms/step - accuracy: 0.0027 - loss: 0.0084
Epoch 2/300
51/51 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step - accuracy: 0.0022 - loss: 0.0073
Epoch 3/300
51/51 ━━━━━━━━━━━━━━━━━━━━ 2s 30ms/step - accuracy: 0.0030 - loss: 0.0073
Epoch 4/300
51/51 ━━━━━━━━━━━━━━━━━━━━ 2s 30ms/step - accuracy: 0.0019 - loss: 0.0073
Epoch 5/300
51/51 ━━━━━━━━━━━━━━━━━━━━ 2s 30ms/step - accuracy: 0.0023 - loss: 0.0072
Epoch 6/300
51/51 ━━━━━━━━━━━━━━━━━━━━ 2s 30ms/step - accuracy: 0.0028 - loss: 0.0072
Epoch 7/300
51/51 ━━━━━━━━━━━━━━━━━━━━ 2s 30ms/step - accuracy: 0.0044 - loss: 0.0076
Epoch 8/300
51/51 ━━━━━━━━━━━━━━━━━━━━ 2s 30ms/step - accuracy: 0.0037 - loss: 0.0073
Epoch 9/300
51/51 ━━━━━━━━━━━━━━━━━━━━ 2s 30ms/step - accuracy: 0.0026 - loss: 0.0075
Epoch 10/300
51/51 ━━━━━━

202/202 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step
75/75 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step
Architecture: rnn
X_train before fit: (6715, 90, 1), y_train: (6715, 1)
Number of 6715 samples, 90 timesteps, 1 features
Epoch 1/300
53/53 ━━━━━━━━━━━━━━━━━━━━ 2s 29ms/step - accuracy: 0.0025 - loss: 0.3381
Epoch 2/300
53/53 ━━━━━━━━━━━━━━━━━━━━ 2s 30ms/step - accuracy: 0.0018 - loss: 0.0109
Epoch 3/300
53/53 ━━━━━━━━━━━━━━━━━━━━ 2s 31ms/step - accuracy: 0.0021 - loss: 0.0106
Epoch 4/300
53/53 ━━━━━━━━━━━━━━━━━━━━ 2s 30ms/step - accuracy: 0.0026 - loss: 0.0090
Epoch 5/300
53/53 ━━━━━━━━━━━━━━━━━━━━ 2s 30ms/step - accuracy: 0.0018 - loss: 0.0082
Epoch 6/300
53/53 ━━━━━━━━━━━━━━━━━━━━ 2s 30ms/step - accuracy: 0.0025 - loss: 0.0083
Epoch 7/300
53/53 ━━━━━━━━━━━━━━━━━━━━ 2s 30ms/step - accuracy: 0.0034 - loss: 0.0084
Epoch 8/300
53/53 ━━━━━━━━━━━━━━━━━━━━ 2s 30ms/step - accuracy: 0.0027 - loss: 0.0083
Epoch 9/300
53/53 ━━━━━━━━━━━━━━━━━━━━ 2s 30ms/step - accuracy: 0.0027 - loss: 0.0073
Epoch 10/300
53/53 ━━━━━━

210/210 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step
67/67 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step
Architecture: rnn
X_train before fit: (6966, 90, 1), y_train: (6966, 1)
Number of 6966 samples, 90 timesteps, 1 features
Epoch 1/300
55/55 ━━━━━━━━━━━━━━━━━━━━ 3s 36ms/step - accuracy: 0.0016 - loss: 0.4163
Epoch 2/300
55/55 ━━━━━━━━━━━━━━━━━━━━ 2s 36ms/step - accuracy: 0.0034 - loss: 0.0126
Epoch 3/300
55/55 ━━━━━━━━━━━━━━━━━━━━ 2s 35ms/step - accuracy: 0.0025 - loss: 0.0103
Epoch 4/300
55/55 ━━━━━━━━━━━━━━━━━━━━ 2s 37ms/step - accuracy: 0.0031 - loss: 0.0086
Epoch 5/300
55/55 ━━━━━━━━━━━━━━━━━━━━ 2s 36ms/step - accuracy: 0.0035 - loss: 0.0093
Epoch 6/300
55/55 ━━━━━━━━━━━━━━━━━━━━ 2s 36ms/step - accuracy: 0.0019 - loss: 0.0089
Epoch 7/300
55/55 ━━━━━━━━━━━━━━━━━━━━ 2s 36ms/step - accuracy: 0.0023 - loss: 0.0081
Epoch 8/300
55/55 ━━━━━━━━━━━━━━━━━━━━ 2s 36ms/step - accuracy: 0.0029 - loss: 0.0083
Epoch 9/300
55/55 ━━━━━━━━━━━━━━━━━━━━ 2s 35ms/step - accuracy: 0.0019 - loss: 0.0081
Epoch 10/300
55/55 ━━━━━━

In [ ]:
index_tickers = [
    "^GSPC",          # S&P 500
    "^IXIC",          # Nasdaq
    "^DJI",           # Dow Jones Industrial Average
    "^FTSE",          # FTSE 100
    "^N225",          # Nikkei 225
    "^IBEX",          # Ibex 35
]

In [ ]:
for ticker in index_tickers:
    ticker_single_models(ticker)